In [12]:
##############
# MUST HAVES #
##############


In [13]:
# Read CSV


In [14]:
# Access API


In [15]:
# Select stocks based on ticker symbol


In [16]:
# Store Stock information as a data set


In [17]:
# Format dataset with classification (up/down)


In [18]:
# Train


In [21]:
# Hidden Nodes


In [20]:
# Dropout


In [22]:
# Test


In [23]:
# Real Life Prediction


In [24]:
# Print results of prediction


In [25]:
#######################
# WOULD LIKE TO HAVES #
#######################


In [26]:
# Save as CSV


In [27]:
# Wait for API (free) access limit


In [28]:
# Save results of Training


In [31]:
# Load Results from Previous Training


In [30]:
# Scrape HTML
